# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
import json

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
csv_path = "../output_data/cities.csv"
city_data = pd.read_csv(csv_path)
city_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Gouré,13.9835,10.2704,80.62,8,42,8.12,NE,1618713612
1,Kurchum,48.5722,83.6542,51.91,54,8,6.78,KZ,1618713613
2,Hermanus,-34.4187,19.2345,71.01,24,0,1.99,ZA,1618713613
3,Kavieng,-2.5744,150.7967,83.55,73,55,9.98,PG,1618713596
4,Bacungan,9.9106,118.7022,84.20,74,90,9.22,PH,1618713614
...,...,...,...,...,...,...,...,...,...
574,Kandiari,26.9156,68.5216,89.33,26,3,8.14,PK,1618713857
575,Mitú,1.1983,-70.1733,71.87,95,100,1.72,CO,1618713858
576,Shalya,57.2471,58.7298,37.26,80,8,8.46,RU,1618713859
577,Huilong,31.8111,121.6550,64.40,37,57,3.71,CN,1618713815


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
gmaps.configure(api_key=g_key)

In [4]:
locations = city_data[["Lat", "Lng"]]
humidity = city_data["Humidity"].astype(float)
max_intensity = city_data["Humidity"].max()
fig = gmaps.figure()
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max_intensity,
                                 point_radius=5)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
clean_data = city_data[(city_data["Max Temp"]>70) &
                        (city_data["Wind Speed"]<5) &
                        (city_data["Cloudiness"]<10)]

clean_data

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
2,Hermanus,-34.4187,19.2345,71.01,24,0,1.99,ZA,1618713613
51,Busselton,-33.6500,115.3333,72.52,44,0,4.79,AU,1618713631
211,Oranjemund,-28.5500,16.4333,73.44,39,5,4.03,NaN,1618713696
292,Benguela,-12.5763,13.4055,79.11,73,2,3.51,AO,1618713729
343,Abu Dhabi,24.4667,54.3667,70.83,73,0,3.44,AE,1618713667
348,Zabid,14.1951,43.3152,76.98,60,7,3.09,YE,1618713755
386,Bārmer,25.7500,71.3833,85.69,26,1,4.34,IN,1618713773
443,Tura,25.5198,90.2201,81.66,50,2,3.62,IN,1618713796
527,Bhātāpāra,21.7333,81.9333,82.40,65,5,3.44,IN,1618713835
542,Dhankutā,26.9833,87.3333,72.09,49,0,3.44,NP,1618713843


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df = clean_data[["City","Lat","Lng","Country"]]

hotel_df

,City,Lat,Lng,Country
2,Hermanus,-34.4187,19.2345,ZA
51,Busselton,-33.6500,115.3333,AU
211,Oranjemund,-28.5500,16.4333,NaN
292,Benguela,-12.5763,13.4055,AO
343,Abu Dhabi,24.4667,54.3667,AE
348,Zabid,14.1951,43.3152,YE
386,Bārmer,25.7500,71.3833,IN
443,Tura,25.5198,90.2201,IN
527,Bhātāpāra,21.7333,81.9333,IN
542,Dhankutā,26.9833,87.3333,NP


In [7]:
api_key=g_key
cities = hotel_df["City"].tolist()
lat = hotel_df["Lat"].tolist()
lon = hotel_df["Lng"].tolist()
hotels=[]

for index, city in enumerate(cities):
    target_coordinates = f"{lat[index]},{lon[index]}"
    target_search = "Hotel"
    target_radius = 5000
    target_type = "hotel"

    params = {
        "location": target_coordinates,
        "keyword": target_search,
        "radius": target_radius,
        "type": target_type,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    response = requests.get(base_url, params=params)

    places_data = response.json()
    try:
        hotels.append(places_data["results"][0]["name"])
        
    except(IndexError):
        hotels.append(0)

hotel_df["Hotel Name"]=hotels
hotel_df = hotel_df[hotel_df["Hotel Name"]!=0]
hotel_df

<ipython-input-7-05e37a84372e>:32: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hotel_df["Hotel Name"]=hotels


,City,Lat,Lng,Country,Hotel Name
2,Hermanus,-34.4187,19.2345,ZA,Harbour House Hotel
51,Busselton,-33.6500,115.3333,AU,Fairlawn Estate
211,Oranjemund,-28.5500,16.4333,NaN,Oryx Guesthouse Oranjemund
292,Benguela,-12.5763,13.4055,AO,Aparthotel Mil Cidades
343,Abu Dhabi,24.4667,54.3667,AE,Emirates Palace
348,Zabid,14.1951,43.3152,YE,دار الضيافة
443,Tura,25.5198,90.2201,IN,Hotel C&C
527,Bhātāpāra,21.7333,81.9333,IN,Hotel Royal Inn
542,Dhankutā,26.9833,87.3333,NP,Hotel Murchunga International Pvt.Ltd
550,Kibala,9.1104,18.3463,TD,hotel safari


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))